## 7.5. Normalización por lotes

Entrenar redes neuronales profundas es difícil. Y hacer que converjan en un tiempo razonable puede ser complicado. En esta sección, describimos la normalización por lotes , una técnica popular y efectiva que acelera de manera constante la convergencia de redes profundas.

Junto con los bloques residuales, cubiertos más adelante en la Sección 7.6 , la normalización por lotes ha hecho posible que los profesionales entrenen rutinariamente redes con más de 100 capas.

### 7.5.1. Entrenamiento de redes profundas
Para motivar la normalización por lotes, revisemos algunos desafíos prácticos que surgen al entrenar modelos de aprendizaje automático y redes neuronales en particular.

En primer lugar, las elecciones relacionadas con el preprocesamiento de datos a menudo marcan una enorme diferencia en los resultados finales. Recuerde nuestra aplicación de MLP para predecir los precios de la vivienda ( Sección 4.10 ). Nuestro primer paso al trabajar con datos reales fue estandarizar nuestras características de entrada para que cada una tuviera una media de cero y una varianza de uno. Intuitivamente, esta estandarización funciona bien con nuestros optimizadores porque coloca los parámetros a priori en una escala similar.

En segundo lugar, para un MLP o CNN típico, a medida que entrenamos, las variables (p. ej., salidas de transformación afines en MLP) en las capas intermedias pueden tomar valores con magnitudes que varían ampliamente: tanto a lo largo de las capas desde la entrada hasta la salida, a través de las unidades en el misma capa, y con el tiempo debido a nuestras actualizaciones de los parámetros del modelo. Los inventores de la normalización por lotes postularon informalmente que esta desviación en la distribución de dichas variables podría dificultar la convergencia de la red. Intuitivamente, podríamos conjeturar que si una capa tiene valores variables que son 100 veces mayores que los de otra capa, esto podría requerir ajustes compensatorios en las tasas de aprendizaje.

En tercer lugar, las redes más profundas son complejas y se pueden sobreadaptar con facilidad. Esto significa que la regularización se vuelve más crítica.

La normalización por lotes se aplica a capas individuales (opcionalmente, a todas ellas) y funciona de la siguiente manera: en cada iteración de entrenamiento, primero normalizamos las entradas (de la normalización por lotes) restando su media y dividiéndola por su desviación estándar, donde ambas se estiman basado en las estadísticas del minilote actual. A continuación, aplicamos un coeficiente de escala y una compensación de escala. Es precisamente debido a esta normalización basada en estadísticas por lotes que la normalización por lotes deriva su nombre.

Tenga en cuenta que si intentáramos aplicar la normalización por lotes con minilotes de tamaño 1, no podríamos aprender nada. ¡Eso es porque después de restar las medias, cada unidad oculta tomaría el valor 0! Como puede suponer, dado que estamos dedicando una sección completa a la normalización de lotes, con minilotes lo suficientemente grandes, el enfoque resulta eficaz y estable. Una conclusión aquí es que al aplicar la normalización por lotes, la elección del tamaño del lote puede ser incluso más importante que sin la normalización por lotes.

Formalmente, denotando por x∈B
 una entrada para la normalización por lotes ( BN
 ) que es de un minilote  B
 , transformaciones de normalización por lotes x
  de acuerdo con la siguiente expresión:

(7.5.1)
BN(x)=γ⊙x−μ^Bσ^B+β.
 
En (7.5.1) , μ^B
  es la media muestral y σ^B
 es la desviación estándar de la muestra del minilote B
 . Después de aplicar la estandarización, el minilote resultante tiene media cero y varianza unitaria. Debido a que la elección de la varianza de la unidad (frente a algún otro número mágico) es una elección arbitraria, comúnmente incluimos el parámetro de escala por elementos  γ
 y cambio de parámetro  β
 que tienen la misma forma que  x
 . Tenga en cuenta que γ
 y  β
 son parámetros que deben aprenderse junto con los demás parámetros del modelo.

En consecuencia, las magnitudes variables para las capas intermedias no pueden divergir durante el entrenamiento porque la normalización por lotes las centra y las vuelve a escalar de forma activa a una media y tamaño dados (a través de  μ^B
 y  σ^B
 ). Una parte de la intuición o sabiduría de los profesionales es que la normalización por lotes parece permitir tasas de aprendizaje más agresivas.

Formalmente, calculamos μ^B
 y  σ^B
 en (7.5.1) como sigue:

(7.5.2)
μ^Bσ^2B=1|B|∑x∈Bx,=1|B|∑x∈B(x−μ^B)2+ϵ.
 
 
Tenga en cuenta que agregamos una pequeña constante ϵ>0
 a la estimación de la varianza para garantizar que nunca intentemos dividir por cero, incluso en los casos en que la estimación de la varianza empírica pueda desaparecer. las estimaciones  μ^B
 y  σ^B
 contrarrestar el problema de escala mediante el uso de estimaciones ruidosas de la media y la varianza. Se podría pensar que este ruido debería ser un problema. Como resultado, esto es realmente beneficioso.

Este resulta ser un tema recurrente en el aprendizaje profundo. Por razones que aún no están bien caracterizadas teóricamente, varias fuentes de ruido en la optimización a menudo conducen a un entrenamiento más rápido y menos sobreajuste: esta variación parece actuar como una forma de regularización. En algunas investigaciones preliminares, [Teye et al., 2018] y [Luo et al., 2018] relacionan las propiedades de la normalización por lotes con los priores bayesianos y las penalizaciones, respectivamente. En particular, esto arroja algo de luz sobre el enigma de por qué la normalización por lotes funciona mejor para tamaños de minilotes moderados en el  50∼100
 rango.

Al arreglar un modelo entrenado, podría pensar que preferiríamos usar el conjunto de datos completo para estimar la media y la varianza. Una vez que se completa el entrenamiento, ¿por qué querríamos que la misma imagen se clasifique de manera diferente, según el lote en el que resida? Durante el entrenamiento, dicho cálculo exacto no es factible porque las variables intermedias para todos los ejemplos de datos cambian cada vez que actualizamos nuestro modelo. Sin embargo, una vez que se entrena el modelo, podemos calcular las medias y las varianzas de las variables de cada capa en función del conjunto de datos completo. De hecho, esta es una práctica estándar para los modelos que emplean la normalización por lotes y, por lo tanto, las capas de normalización por lotes funcionan de manera diferente en el modo de entrenamiento (normalización por estadísticas de minilotes) y en el modo de predicción (normalización por estadísticas de conjuntos de datos).

Ahora estamos listos para ver cómo funciona la normalización por lotes en la práctica.

### 7.5.2. Capas de normalización por lotes
Las implementaciones de normalización por lotes para capas totalmente conectadas y capas convolucionales son ligeramente diferentes. Discutimos ambos casos a continuación. Recuerde que una diferencia clave entre la normalización por lotes y otras capas es que debido a que la normalización por lotes opera en un minilote completo a la vez, no podemos simplemente ignorar la dimensión del lote como lo hicimos antes al introducir otras capas.

#### 7.5.2.1. Capas totalmente conectadas
Al aplicar la normalización por lotes a capas completamente conectadas, el documento original inserta la normalización por lotes después de la transformación afín y antes de la función de activación no lineal (las aplicaciones posteriores pueden insertar la normalización por lotes justo después de las funciones de activación) [Ioffe & Szegedy, 2015 ] . Denotando la entrada a la capa completamente conectada por x
 , la transformación afín por Wx+b
 (con el parámetro de peso  W
 y el parámetro de sesgo b
 ), y la función de activación por ϕ
 , podemos expresar el cálculo de una salida de capa totalmente conectada y habilitada para la normalización por lotes  h
 como sigue:

(7.5.3)
h=ϕ(BN(Wx+b)).
 
Recuerde que la media y la varianza se calculan en el mismo minilote en el que se aplica la transformación.

#### 7.5.2.2. Capas convolucionales
De manera similar, con las capas convolucionales, podemos aplicar la normalización por lotes después de la convolución y antes de la función de activación no lineal. Cuando la convolución tiene múltiples canales de salida, necesitamos llevar a cabo una normalización por lotes para cada una de las salidas de estos canales, y cada canal tiene su propia escala y parámetros de cambio, los cuales son escalares. Supongamos que nuestros minilotes contienen m
 ejemplos y que para cada canal, la salida de la convolución tiene altura p
 y ancho q
 . Para las capas convolucionales, llevamos a cabo la normalización de cada lote sobre el m⋅p⋅q
 elementos por canal de salida simultáneamente. Por lo tanto, recopilamos los valores de todas las ubicaciones espaciales al calcular la media y la varianza y, en consecuencia, aplicamos la misma media y la misma varianza dentro de un canal determinado para normalizar el valor en cada ubicación espacial.

#### 7.5.2.3. Normalización de lotes durante la predicción
Como mencionamos anteriormente, la normalización por lotes generalmente se comporta de manera diferente en el modo de entrenamiento y el modo de predicción. Primero, el ruido en la media de la muestra y la varianza de la muestra que surge de estimar cada uno en minilotes ya no son deseables una vez que hemos entrenado el modelo. En segundo lugar, es posible que no tengamos el lujo de calcular estadísticas de normalización por lote. Por ejemplo, es posible que necesitemos aplicar nuestro modelo para hacer una predicción a la vez.

Por lo general, después del entrenamiento, usamos todo el conjunto de datos para calcular estimaciones estables de las estadísticas variables y luego las fijamos en el momento de la predicción. En consecuencia, la normalización por lotes se comporta de manera diferente durante el entrenamiento y en el momento de la prueba. Recuérdese que la deserción también exhibe esta característica.

## 7.5.3. Implementación desde cero
A continuación, implementamos una capa de normalización por lotes con tensores desde cero.

In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use AI::MXNet qw(mx);
use d2l;
use d2l::Vocab;
use d2l::SeqDataLoader;
use d2l::Timer;
IPerl->load_plugin('Chart::Plotly'); # Jupyter
#import Chart::Plotly 'show_plot'; # localmente
use List::Util qw(min shuffle);


In [9]:
use strict;
use warnings;
use AI::MXNet qw(:all);

sub batch_norm {
    my ($X, $gamma, $beta, $moving_mean, $moving_var, $eps, $momentum) = @_;
    
    # Use `autograd` to determine whether the current mode is training mode or
    # prediction mode
    if (not autograd->is_training()) {
        # If it is prediction mode, directly use the mean and variance
        # obtained by moving average
        my $X_hat = ($X - $moving_mean) / sqrt($moving_var + $eps);
        return ($gamma * $X_hat + $beta, $moving_mean, $moving_var);
    }
    
    assert($X->ndim == 2 or $X->ndim == 4);
    my ($mean, $var);
    if ($X->ndim == 2) {
        # When using a fully-connected layer, calculate the mean and
        # variance on the feature dimension
        $mean = $X->mean(axis => 0);
        $var = (($X - $mean)**2)->mean(axis => 0);
    } else {
        # When using a two-dimensional convolutional layer, calculate the
        # mean and variance on the channel dimension (axis=1). Here we
        # need to maintain the shape of `X`, so that the broadcasting
        # operation can be carried out later
        $mean = $X->mean(axis => [0, 2, 3], keepdims => 1);
        $var = (($X - $mean)**2)->mean(axis => [0, 2, 3], keepdims => 1);
    }
    # In training mode, the current mean and variance are used for the
    # standardization
    my $X_hat = ($X - $mean) / sqrt($var + $eps);
    # Update the mean and variance using moving average
    $moving_mean = $momentum * $moving_mean + (1.0 - $momentum) * $mean;
    $moving_var = $momentum * $moving_var + (1.0 - $momentum) * $var;
    return ($gamma * $X_hat + $beta, $moving_mean, $moving_var);
}

Warning: Subroutine batch_norm redefined at reply input line 5.


Ahora podemos crear una BatchNormcapa adecuada. Nuestra capa mantendrá los parámetros adecuados para la escala gammay el cambio beta, los cuales se actualizarán en el transcurso del entrenamiento. Además, nuestra capa mantendrá los promedios móviles de las medias y las varianzas para su uso posterior durante la predicción del modelo.

Dejando a un lado los detalles algorítmicos, tenga en cuenta el patrón de diseño que subyace a nuestra implementación de la capa. Por lo general, definimos las matemáticas en una función separada, digamosbatch_norm. Luego integramos esta funcionalidad en una capa personalizada, cuyo código se ocupa principalmente de cuestiones de contabilidad, como mover datos al contexto del dispositivo correcto, asignar e inicializar cualquier variable requerida, realizar un seguimiento de los promedios móviles (aquí para la media y la varianza), etc. . Este patrón permite una clara separación de las matemáticas del código repetitivo. También tenga en cuenta que, por conveniencia, no nos preocupamos por inferir automáticamente la forma de entrada aquí, por lo tanto, debemos especificar la cantidad de características en todo momento. No se preocupe, las API de normalización por lotes de alto nivel en el marco de aprendizaje profundo se encargarán de esto por nosotros y lo demostraremos más adelante.

In [10]:
use AI::MXNet qw(mx);
use AI::MXNet::Function::Parameters;

package BatchNorm;
use Mouse;

has 'num_features' => (is => 'ro', isa => 'Int', required => 1);
has 'num_dims' => (is => 'ro', isa => 'Int', required => 1);
has 'gamma' => (is => 'ro', isa => 'AI::MXNet::NDArray', builder => '_build_gamma', lazy => 1);
has 'beta' => (is => 'ro', isa => 'AI::MXNet::NDArray', builder => '_build_beta', lazy => 1);
has 'moving_mean' => (is => 'rw', isa => 'AI::MXNet::NDArray', builder => '_build_moving_mean', lazy => 1);
has 'moving_var' => (is => 'rw', isa => 'AI::MXNet::NDArray', builder => '_build_moving_var', lazy => 1);

sub _build_gamma {
    my $self = shift;
    my $shape = $self->num_dims == 2 ? [1, $self->num_features] : [1, $self->num_features, 1, 1];
    return AI::MXNet::NDArray->zeros($shape) + 1;
}

sub _build_beta {
    my $self = shift;
    my $shape = $self->num_dims == 2 ? [1, $self->num_features] : [1, $self->num_features, 1, 1];
    return AI::MXNet::NDArray->zeros($shape);
}

sub _build_moving_mean {
    my $self = shift;
    my $shape = $self->num_dims == 2 ? [1, $self->num_features] : [1, $self->num_features, 1, 1];
    return AI::MXNet::NDArray->zeros($shape);
}

sub _build_moving_var {
    my $self = shift;
    my $shape = $self->num_dims == 2 ? [1, $self->num_features] : [1, $self->num_features, 1, 1];
    return AI::MXNet::NDArray->ones($shape);
}

sub forward {
    my ($self, $X) = @_;
    # If `X` is not on the main memory, copy `moving_mean` and
    # `moving_var` to the device where `X` is located
    if ($self->moving_mean->context ne $X->context) {
        $self->moving_mean($self->moving_mean->copyto($X->context));
        $self->moving_var($self->moving_var->copyto($X->context));
    }
    # Save the updated `moving_mean` and `moving_var`
    my ($Y, $moving_mean, $moving_var) = batch_norm(
        $X, $self->gamma->aspdl, $self->beta->aspdl, $self->moving_mean->aspdl,
        $self->moving_var->aspdl, eps=>1e-12, momentum=>0.9);
    $self->moving_mean($moving_mean);
    $self->moving_var($moving_var);
    return $Y;
}

1;

1

Warning: Subroutine _build_gamma redefined at reply input line 14.

Subroutine _build_beta redefined at reply input line 20.

Subroutine _build_moving_mean redefined at reply input line 26.

Subroutine _build_moving_var redefined at reply input line 32.

Subroutine forward redefined at reply input line 38.


## 7.5.4. Aplicación de la normalización por lotes en LeNet
Para ver cómo aplicar BatchNormen contexto, a continuación lo aplicamos a un modelo LeNet tradicional ( Sección 6.6 ). Recuerde que la normalización por lotes se aplica después de las capas convolucionales o capas totalmente conectadas, pero antes de las funciones de activación correspondientes.


In [18]:
use AI::MXNet qw(mx);
use AI::MXNet::Gluon qw(gluon);

my $net = gluon->nn->Sequential();
$net->add(gluon->nn->Conv2D(6, kernel_size=>5));
$net->add(gluon->contrib->BatchNorm(6, num_dims=>4));
$net->add(gluon->nn->Activation('sigmoid'));
$net->add(gluon->nn->MaxPool2D(pool_size=>2, strides=>2));
$net->add(gluon->nn->Conv2D(16, kernel_size=>5));
$net->add(gluon->contrib->BatchNorm(16, num_dims=>4));
$net->add(gluon->nn->Activation('sigmoid'));
$net->add(gluon->nn->MaxPool2D(pool_size=>2, strides=>2));
$net->add(gluon->nn->Dense(120));
$net->add(gluon->contrib->BatchNorm(120, num_dims=>2));
$net->add(gluon->nn->Activation('sigmoid'));
$net->add(gluon->nn->Dense(84));
$net->add(gluon->contrib->BatchNorm(84, num_dims=>2));
$net->add(gluon->nn->Activation('sigmoid'));
$net->add(gluon->nn->Dense(10));


Warning: Can't locate object method "contrib" via package "AI::MXNet::Gluon" at reply input line 6.

Use of uninitialized value in goto at (eval 992) line 9.


Error: goto must have label at (eval 992) line 9.


Como antes, entrenaremos nuestra red en el conjunto de datos Fashion-MNIST. Este código es prácticamente idéntico al que usamos cuando entrenamos LeNet por primera vez ( Sección 6.6 ). La principal diferencia es la tasa de aprendizaje considerablemente mayor.

In [19]:
use AI::MXNet qw(mx);

my $lr = 1.0;
my $num_epochs = 10;
my $batch_size = 256;

my ($train_iter, $test_iter) = mx->test_utils->get_mnist_iter(
    batch_size => $batch_size,
    input_shape => [1, 28, 28],
    data_type => 'float32',
    flat => 0,
);

my $ctx = mx->gpu(0); # or mx->cpu()
my $net = d2l->get_net();
$net->initialize(ctx => $ctx);

d2l->train_ch6($net, $train_iter, $test_iter, $num_epochs, $lr, $ctx);

Warning: Can't locate object method "test_utils" via package "AI::MXNet" at reply input line 7.

Use of uninitialized value in goto at (eval 950) line 9.


Error: goto must have label at (eval 950) line 9.


Echemos un vistazo al parámetro de escala gammay el parámetro de cambio betaaprendido de la primera capa de normalización por lotes.

In [21]:
use AI::MXNet qw(mx);

my $net = ...; # Tu red neuronal definida aquí
my $layer = 1; # Índice de la capa que quieres acceder

my $gamma = $net->get_params()[2*$layer];
$gamma = $gamma->reshape([-1]);

my $beta = $net->get_params()[2*$layer + 1];
$beta = $beta->reshape([-1]);

Warning: Variable "$gamma" is not imported at reply input line 7.

	(Did you mean &gamma instead?)

Variable "$gamma" is not imported at reply input line 7.

	(Did you mean &gamma instead?)

Variable "$beta" is not imported at reply input line 10.

	(Did you mean &beta instead?)

Variable "$beta" is not imported at reply input line 10.

	(Did you mean &beta instead?)


Error: syntax error at reply input line 3, near "= ..."
syntax error at reply input line 6, near ")["
Global symbol "$gamma" requires explicit package name (did you forget to declare "my $gamma"?) at reply input line 7.
Global symbol "$gamma" requires explicit package name (did you forget to declare "my $gamma"?) at reply input line 7.
syntax error at reply input line 9, near ")["
Global symbol "$beta" requires explicit package name (did you forget to declare "my $beta"?) at reply input line 10.
Global symbol "$beta" requires explicit package name (did you forget to declare "my $beta"?) at reply input line 10.
BEGIN not safe after errors--compilation aborted at reply input line 14.



## 7.5.5. Implementación concisa
En comparación con la BatchNormclase, que acabamos de definir nosotros mismos, podemos usar la BatchNormclase definida en las API de alto nivel directamente desde el marco de aprendizaje profundo. El código se ve prácticamente idéntico a la aplicación de nuestra implementación anterior.

In [20]:
net = nn.Sequential()
net.add(nn.Conv2D(6, kernel_size=5), nn.BatchNorm(), nn.Activation('sigmoid'),
        nn.MaxPool2D(pool_size=2, strides=2), nn.Conv2D(16, kernel_size=5),
        nn.BatchNorm(), nn.Activation('sigmoid'),
        nn.MaxPool2D(pool_size=2, strides=2), nn.Dense(120), nn.BatchNorm(),
        nn.Activation('sigmoid'), nn.Dense(84), nn.BatchNorm(),
        nn.Activation('sigmoid'), nn.Dense(10))

Warning: Semicolon seems to be missing at reply input line 1.


Error: Can't modify constant item in scalar assignment at reply input line 2, near ")
net"
syntax error at reply input line 2, near ")
net"
BEGIN not safe after errors--compilation aborted at reply input line 11.



A continuación, usamos los mismos hiperparámetros para entrenar nuestro modelo. Tenga en cuenta que, como de costumbre, la variante de API de alto nivel se ejecuta mucho más rápido porque su código se ha compilado en C++ o CUDA, mientras que Python debe interpretar nuestra implementación personalizada.

In [22]:
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

Error: Bareword "d2l" not allowed while "strict subs" in use at reply input line 2.
BEGIN not safe after errors--compilation aborted at reply input line 5.



## 7.5.6. Controversia
Intuitivamente, se cree que la normalización por lotes hace que el panorama de optimización sea más fluido. Sin embargo, debemos tener cuidado de distinguir entre intuiciones especulativas y explicaciones verdaderas de los fenómenos que observamos cuando entrenamos modelos profundos. Recuerde que ni siquiera sabemos por qué las redes neuronales profundas más simples (MLP y CNN convencionales) se generalizan bien en primer lugar. Incluso con la deserción escolar y la disminución del peso, siguen siendo tan flexibles que su capacidad para generalizar a datos no vistos no puede explicarse a través de garantías de generalización teóricas de aprendizaje convencionales.

En el artículo original que proponía la normalización por lotes, los autores, además de presentar una herramienta poderosa y útil, ofrecieron una explicación de por qué funciona: al reducir el cambio de covariable interna . Presumiblemente, por cambio de covariable interna los autores se referían a algo parecido a la intuición expresada anteriormente: la noción de que la distribución de los valores de las variables cambia durante el transcurso del entrenamiento. Sin embargo, hubo dos problemas con esta explicación: i) Esta deriva es muy diferente del cambio de covariable , lo que hace que el nombre sea inapropiado. ii) La explicación ofrece una intuición subespecificada pero deja la pregunta de por qué precisamente esta técnica funcionauna pregunta abierta que necesita una explicación rigurosa. A lo largo de este libro, nuestro objetivo es transmitir las intuiciones que utilizan los profesionales para guiar el desarrollo de redes neuronales profundas. Sin embargo, creemos que es importante separar estas intuiciones rectoras de los hechos científicos establecidos. Eventualmente, cuando domine este material y comience a escribir sus propios trabajos de investigación, querrá ser claro para delinear entre afirmaciones técnicas y corazonadas.

Tras el éxito de la normalización por lotes, su explicación en términos de cambio de covariable interno ha surgido repetidamente en debates en la literatura técnica y en un discurso más amplio sobre cómo presentar la investigación de aprendizaje automático. En un discurso memorable pronunciado al aceptar un premio Test of Time en la conferencia NeurIPS de 2017, Ali Rahimi utilizó el cambio de covariable interno como punto focal en un argumento que comparaba la práctica moderna del aprendizaje profundo con la alquimia. Posteriormente, el ejemplo se revisó en detalle en un documento de posición que describe tendencias preocupantes en el aprendizaje automático [Lipton & Steinhardt, 2018]. Otros autores han propuesto explicaciones alternativas para el éxito de la normalización por lotes, y algunos afirman que el éxito de la normalización por lotes se produce a pesar de exhibir un comportamiento que, en cierto modo, es opuesto al que se afirma en el artículo original [Santurkar et al., 2018 ] .

Observamos que el cambio de covariable interno no es más digno de crítica que cualquiera de las miles de afirmaciones igualmente vagas que se hacen cada año en la literatura técnica sobre aprendizaje automático. Probablemente, su resonancia como punto focal de estos debates se debe a su amplio reconocimiento por parte del público objetivo. La normalización por lotes ha demostrado ser un método indispensable, aplicado en casi todos los clasificadores de imágenes implementados, lo que le valió al artículo que introdujo la técnica decenas de miles de citas.

## 7.5.7. Resumen
Durante el entrenamiento del modelo, la normalización por lotes ajusta continuamente la salida intermedia de la red neuronal utilizando la media y la desviación estándar del minilote, de modo que los valores de la salida intermedia en cada capa de la red neuronal sean más estables.

Los métodos de normalización por lotes para capas totalmente conectadas y capas convolucionales son ligeramente diferentes.

Al igual que una capa de exclusión, las capas de normalización por lotes tienen resultados de cálculo diferentes en el modo de entrenamiento y el modo de predicción.

La normalización por lotes tiene muchos efectos secundarios beneficiosos, principalmente el de la regularización. Por otro lado, la motivación original de reducir el cambio de covariable interno parece no ser una explicación válida.